# Rating Prediction of Restaurants

### Importing Required libraries

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Loading the dataset into Dataframe
df=pd.read_csv('zomato.csv')

In [3]:
#Eyeballing the data
df

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [5]:
#Dropping irrelevant columns
df1=df.drop(['url','menu_item','reviews_list','phone','name','address','dish_liked'],axis=1)
df1.isnull().sum()

online_order                      0
book_table                        0
rate                           7775
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     346
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

In [6]:
#Dropping miising values since they are very low in perncentage
df1.dropna(inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43533 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   online_order                 43533 non-null  object
 1   book_table                   43533 non-null  object
 2   rate                         43533 non-null  object
 3   votes                        43533 non-null  int64 
 4   location                     43533 non-null  object
 5   rest_type                    43533 non-null  object
 6   cuisines                     43533 non-null  object
 7   approx_cost(for two people)  43533 non-null  object
 8   listed_in(type)              43533 non-null  object
 9   listed_in(city)              43533 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.7+ MB


In [7]:
#Pre-processing the 'rate' column and converting its type from object to int
df1.rate = df1.rate.replace("-", np.nan)
df1.rate = df1.rate.replace("NEW", np.nan)
df1.dropna(how ='any', inplace = True)
df1.rate = df1.rate.astype(str)
df1.rate = df1.rate.apply(lambda x: x.replace('/5',''))
df1.rate = df1.rate.apply(lambda x: float(x))
df1.head()

,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),listed_in(type),listed_in(city)
0,Yes,Yes,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,Buffet,Banashankari
1,Yes,No,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,Buffet,Banashankari
2,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,Buffet,Banashankari
3,No,No,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300,Buffet,Banashankari
4,No,No,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,Buffet,Banashankari


In [8]:
df1.online_order.replace(('Yes','No'),(1, 0),inplace=True)
df1.book_table.replace(('Yes','No'),(1, 0),inplace=True)

In [9]:
df2=df1.copy()

In [10]:
#Encoding categorical data 
def Encode(df2):
    for column in df2.columns[~df2.columns.isin(['rate', 'approx_cost(for two people)', 'votes'])]:
        df2[column] = df2[column].factorize()[0]
    return df2

df2_en = Encode(df2.copy())

In [11]:
df2_en = df2_en.rename(columns={'approx_cost(for two people)':'cost'})

In [12]:
df2_en['cost'] = df2_en['cost'].astype(str)
df2_en['cost'] = df2_en['cost'].apply(lambda x: x.replace(',','.'))
df2_en['cost'] = df2_en['cost'].astype(float)
df2_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41263 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   online_order     41263 non-null  int64  
 1   book_table       41263 non-null  int64  
 2   rate             41263 non-null  float64
 3   votes            41263 non-null  int64  
 4   location         41263 non-null  int64  
 5   rest_type        41263 non-null  int64  
 6   cuisines         41263 non-null  int64  
 7   cost             41263 non-null  float64
 8   listed_in(type)  41263 non-null  int64  
 9   listed_in(city)  41263 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 3.5 MB


In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
df2_en[['online_order', 'book_table', 'votes', 'rest_type',
       'cuisines', 'cost', 'listed_in(type)',
       'listed_in(city)']] = mms.fit_transform(df2_en[['online_order', 'book_table', 'votes', 'rest_type',
       'cuisines', 'cost', 'listed_in(type)',
       'listed_in(city)']])
df2_en.head()

In [21]:
X = df2_en.iloc[:, [0,1,3,4,5,6,7,8]].values
y = df2_en.iloc[:, 2].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0, stratify = y)


In [13]:
from sklearn.linear_model import LogisticRegression #Logistic Regression is a Machine Learning classification algorithm
from sklearn.linear_model import LinearRegression #Linear Regression is a Machine Learning classification algorithm
from sklearn.model_selection import train_test_split #Splitting of Dataset
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
reg=LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.28800520690863163

In [22]:
from sklearn.ensemble import RandomForestRegressor
RForest=RandomForestRegressor(n_estimators=500,random_state=150,min_samples_leaf=.0001)
RForest.fit(X_train,y_train)
y_predict=RForest.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

0.8700451689990917

In [23]:
import pickle
file=open('Model.pkl','wb')
pickle.dump(RForest, file)